# Compose

Composition is the art of chaining function calls for a desire outcome
Suppose the following functions

- $f(x): x + 2$
- $g(x): x \times 10$
- $h(x): x \div 5$

## Aid
```python

_f = lambda x: x + 2
_g = lambda x: x * 10
_h = lambda x: x / 5

from toolz import compose
_z = compose(_h, _g, _f)

list(map(_z, range(10)))
```

> __Alternative:__ $ (x + 2)\frac{10}{5} $


# Curry

Breaking a function that takes multiple arguments into a series of functions that take `n` arguments

```python
from toolz import curry

@curry
def mysum(x,y):
    return x + y

mysum(5,5)

# What happen with: 
# mysum(5)

```


# Exercise #2

I want to:
- Filter numbers from `MX` Mexico
- Replace all `-` by dots `.`
- Prepend country code `+52.` to the numbers

In [129]:
data = [
    {"MX" : "111-222-333"},
    {"MX" : "444-555-666"},
    {"NL" : "999-000-000"},
    {"FR" : "555-555-555"}
]
data

[{'MX': '111-222-333'},
 {'MX': '444-555-666'},
 {'NL': '999-000-000'},
 {'FR': '555-555-555'}]

### For-Loops

In [130]:
def convert_loop(data):
    new_phones = []
    for d in data:
        for k,v in d.items():
            if k == "MX":
                new_phone = "+52." + v.replace("-", ".")
                new_phones.append({"MX" : new_phone})

    return new_phones


### Compose-Curry

In [ ]:
from toolz import curried as C
from toolz import compose
from operator import add
from functools import partial
from operator import eq

def convert_map(data):
    _is_mx = C.keyfilter(partial(eq, 'MX'))
    _dot = lambda x: x.replace("-", ".")
    _code = partial(add, "+52.")
    _fix_number = C.valmap(compose(_code, _dot))
    return list(map(_fix_number, filter(_is_mx, data)))

## Proof

In [113]:
import ast

# Example Python code as a string
code = """
new_phones = []
for d in data:
    for k,v in d.items():
        if k == "MX":
            new_phone = "+52." + v.replace("-", ".")
            new_phones.append({"MX" : new_phone})
"""

# Parse to AST
tree = ast.parse(code)

# Count instruction-like nodes (statements)
instruction_types = (
    ast.Assign, ast.AugAssign, ast.AnnAssign,
    ast.For, ast.If, ast.While, ast.With, ast.Expr,
    ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef,
    ast.Return, ast.Raise, ast.Try, ast.Assert, ast.Import,
    ast.ImportFrom, ast.Delete, ast.Global, ast.Nonlocal, ast.Pass,
    ast.Break, ast.Continue
)

# Walk the tree and count
instruction_count = sum(1 for node in ast.walk(tree) if isinstance(node, instruction_types))

print("Number of instructions:", instruction_count)


Number of instructions: 6


In [114]:
import ast

# Example Python code as a string
code = """
_is_mx = C.keyfilter(partial(eq, 'MX'))
_dot = lambda x: x.replace("-", ".")
_code = partial(add, "+52.")
_fix_number = C.valmap(compose(_code, _dot))
"""

# Parse to AST
tree = ast.parse(code)

# Count instruction-like nodes (statements)
instruction_types = (
    ast.Assign, ast.AugAssign, ast.AnnAssign,
    ast.For, ast.If, ast.While, ast.With, ast.Expr,
    ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef,
    ast.Return, ast.Raise, ast.Try, ast.Assert, ast.Import,
    ast.ImportFrom, ast.Delete, ast.Global, ast.Nonlocal, ast.Pass,
    ast.Break, ast.Continue
)


# Walk the tree and count
instruction_count = sum(1 for node in ast.walk(tree) if isinstance(node, instruction_types))

print("Number of instructions:", instruction_count)

Number of instructions: 4


In [117]:
import dis

dis.dis(convert_loop)

  1           0 RESUME                   0

  2           2 BUILD_LIST               0
              4 STORE_FAST               1 (new_phones)

  3           6 LOAD_FAST                0 (data)
              8 GET_ITER
        >>   10 FOR_ITER                72 (to 158)
             14 STORE_FAST               2 (d)

  4          16 LOAD_FAST                2 (d)
             18 LOAD_ATTR                1 (NULL|self + items)
             38 CALL                     0
             46 GET_ITER
        >>   48 FOR_ITER                51 (to 154)
             52 UNPACK_SEQUENCE          2
             56 STORE_FAST               3 (k)
             58 STORE_FAST               4 (v)

  5          60 LOAD_FAST                3 (k)
             62 LOAD_CONST               1 ('MX')
             64 COMPARE_OP              40 (==)
             68 POP_JUMP_IF_TRUE         1 (to 72)
             70 JUMP_BACKWARD           12 (to 48)

  6     >>   72 LOAD_CONST               2 ('+52.')
             

In [116]:
import dis

dis.dis(convert_map)

  7           0 RESUME                   0

  8           2 LOAD_GLOBAL              0 (C)
             12 LOAD_ATTR                3 (NULL|self + keyfilter)
             32 LOAD_GLOBAL              5 (NULL + partial)
             42 LOAD_GLOBAL              6 (eq)
             52 LOAD_CONST               1 ('MX')
             54 CALL                     2
             62 CALL                     1
             70 STORE_FAST               1 (_is_mx)

  9          72 LOAD_CONST               2 (<code object <lambda> at 0x78a5f2afbb40, file "/tmp/ipykernel_15398/3165421500.py", line 9>)
             74 MAKE_FUNCTION            0
             76 STORE_FAST               2 (_dot)

 10          78 LOAD_GLOBAL              5 (NULL + partial)
             88 LOAD_GLOBAL              8 (add)
             98 LOAD_CONST               3 ('+52.')
            100 CALL                     2
            108 STORE_FAST               3 (_code)

 11         110 LOAD_GLOBAL              0 (C)
           

In [127]:
import opcode

for op in convert_map.__code__.co_code:
    print(op, opcode.opname[op])

151 RESUME
0 CACHE
116 LOAD_GLOBAL
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
106 LOAD_ATTR
3 INTERPRETER_EXIT
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
116 LOAD_GLOBAL
5 END_SEND
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
116 LOAD_GLOBAL
6 <6>
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
100 LOAD_CONST
1 POP_TOP
171 CALL
2 PUSH_NULL
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
171 CALL
1 POP_TOP
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
125 STORE_FAST
1 POP_TOP
100 LOAD_CONST
2 PUSH_NULL
132 MAKE_FUNCTION
0 CACHE
125 STORE_FAST
2 PUSH_NULL
116 LOAD_GLOBAL
5 END_SEND
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
116 LOAD_GLOBAL
8 <8>
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
100 LOAD_CONST
3 INTERPRETER_EXIT
171 CALL
2 PUSH_NULL
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0 CACHE
0